## Занятие 1. Обратное распространение ошибки + Numpy туториал. 

### Запуск тестов пакета

In [2]:
from a4_course_cvdl_t1.relu import ReluLayer

Если указать конкретный модуль с тестами - будет запущен только он

Если добавить флаг `-x` - после первого падения тесты будут прерваны.

In [3]:
! pytest --pyargs a4_course_cvdl_t1.tests.test_0relu -x

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: D:\edu\teach\course_cvdl\classes\c01
plugins: anyio-3.3.2
collected 4 items

. F

================================== FAILURES ===================================
_______________________ TestLayer.test_init[test_data0] _______________________

self = <a4_course_cvdl_t1.tests.base.TestLayer object at 0x0000013570490910>
test_data = {'meta': {'description': 'The only ReLU test.', 'layer_kwargs': {}}, 'values': {'input': [[[[0.3445909917354584, -0.44...36354065], [0.320019006729126, 0.7765920162200928, 0.8497490286827087, 0.796563982963562, 0.9298689961433411]]]], ...}}
test_layer_cls = <class 'a4_course_cvdl_t1.relu.ReluLayer'>

    def test_init(self, test_data, test_layer_cls):
        values = test_data['values']
        meta = test_data['meta']
        layer_kwargs = meta['layer_kwargs']
    
        # check layer creation
     

In [4]:
import numpy as np
from a4_course_cvdl_t1.relu import ReluLayer
print(help(ReluLayer))

Help on class ReluLayer in module a4_course_cvdl_t1.relu:

class ReluLayer(a4_course_cvdl_t1.base.BaseLayer)
 |  Слой, выполняющий Relu активацию y = max(x, 0).
 |  Не имеет параметров.
 |  
 |  Method resolution order:
 |      ReluLayer
 |      a4_course_cvdl_t1.base.BaseLayer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  backward(self, output_grad: numpy.ndarray) -> numpy.ndarray
 |      Обратный проход, принимающий градиент ошибки по выходному тензору,
 |       (результату метода .forward) и возвращающий градиент ошибки по
 |       входному тензору (аргумент метода .forward).
 |      Метод .backward всегда вызывается после одного вызова .forward.
 |      Метод .backward должен записать в .parameters_grads градиенты
 |       параметров.
 |      Слои принимают и возвращают только один тензор.
 |  
 |  forward(self, input: numpy.ndarray) -> numpy.ndarray
 |      Прямой прохо

### Исправляем ReluLayer.1
Задача: дописать ReluLayer.\_\_init\_\_, .forward()

### Создание тензоров

In [5]:
a = np.random.rand(2, 3, 4) - 0.5
b = np.zeros(a.shape)
print(a) 
print(b)

[[[-0.35068074 -0.05131821  0.47700219  0.29850093]
  [-0.11914353 -0.20530368 -0.33724537 -0.39571214]
  [ 0.14459687 -0.43430318  0.35665934 -0.08216197]]

 [[ 0.11320301  0.31408513 -0.28550827 -0.20182455]
  [-0.37239303  0.15515673  0.2251532   0.13596592]
  [-0.26879916 -0.33638794  0.4108995   0.25331197]]]
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


### Поиск максимума

In [6]:
np.max(a)

0.4770021890947693

In [7]:
np.maximum(a, b)

array([[[0.        , 0.        , 0.47700219, 0.29850093],
        [0.        , 0.        , 0.        , 0.        ],
        [0.14459687, 0.        , 0.35665934, 0.        ]],

       [[0.11320301, 0.31408513, 0.        , 0.        ],
        [0.        , 0.15515673, 0.2251532 , 0.13596592],
        [0.        , 0.        , 0.4108995 , 0.25331197]]])

In [8]:
np.argmax(a)

2

Argmax возвращает "плоский" индекс, его нельзя использовать напрямую для индексации многомерного массива - получится
**IndexError**

In [9]:
print(a.flatten()[np.argmax(a)])
print('---IndexError----')
a[np.argmax(a)]

0.4770021890947693
---IndexError----


IndexError: index 2 is out of bounds for axis 0 with size 2

"Плоский" индекс можно раскрутитьв N-мерный индекс, и использовать его для индексации

In [10]:
idx_nd = np.unravel_index(np.argmax(a), a.shape)
print(idx_nd)
a[idx_nd]

(0, 0, 2)


0.4770021890947693

### Поэлементные операции

In [11]:
a * b

array([[[-0., -0.,  0.,  0.],
        [-0., -0., -0., -0.],
        [ 0., -0.,  0., -0.]],

       [[ 0.,  0., -0., -0.],
        [-0.,  0.,  0.,  0.],
        [-0., -0.,  0.,  0.]]])

In [12]:
a + b

array([[[-0.35068074, -0.05131821,  0.47700219,  0.29850093],
        [-0.11914353, -0.20530368, -0.33724537, -0.39571214],
        [ 0.14459687, -0.43430318,  0.35665934, -0.08216197]],

       [[ 0.11320301,  0.31408513, -0.28550827, -0.20182455],
        [-0.37239303,  0.15515673,  0.2251532 ,  0.13596592],
        [-0.26879916, -0.33638794,  0.4108995 ,  0.25331197]]])

### Срезы

In [13]:
print("a             \t", a.shape)
print("a[0]          \t", a[0].shape)
print("a[:, 0]       \t", a[:, 0].shape)
print("a[:, 0, 1:2]  \t", a[:, 0, 0:2].shape)
print("a[:, [0], 1:2]\t", a[:, [0], 0:2].shape)
print("a[:, None]    \t", a[:, None].shape)

a             	 (2, 3, 4)
a[0]          	 (3, 4)
a[:, 0]       	 (2, 4)
a[:, 0, 1:2]  	 (2, 2)
a[:, [0], 1:2]	 (2, 1, 2)
a[:, None]    	 (2, 1, 3, 4)


### Индексация
Булевая индексация: индекс имеет форму тензора со значениями True и False

In [14]:
idx = a > 0
print(a.shape, idx.shape, idx.dtype)
idx

(2, 3, 4) (2, 3, 4) bool


array([[[False, False,  True,  True],
        [False, False, False, False],
        [ True, False,  True, False]],

       [[ True,  True, False, False],
        [False,  True,  True,  True],
        [False, False,  True,  True]]])

Результат индексации - плоский массив

In [15]:
a[idx].shape

(11,)

In [16]:
a[a == np.max(a)]

array([0.47700219])

Индексация "координатами": индекс - массивы N-мерных координат элементов

In [17]:
# Создадим пробный тензор для индексации
c = np.arange(
    np.prod(a.shape)
).reshape(a.shape)
print(c.shape)
c

(2, 3, 4)


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [20]:
# Каждый столбец - индексирует один элемент
# Результат - плоский список
c[
    [0,  1,  0,  0,  0,  0],
    [1, -1,  2, -1, -1, -1],
    [3,  2,  1,  0,  0,  0]
]

array([ 7, 22,  9,  8,  8,  8])

In [21]:
# Если индексы - не плоские, то и результат не плоский
c[
    [[0,  1,  0],[  0,  0,  0]],
    [[1, -1,  2],[ -1, -1, -1]],
    [[3,  2,  1],[  0, 0,   0]],
]

array([[ 7, 22,  9],
       [ 8,  8,  8]])

### Исправляем ReluLayer.2
Задача: дописать ReluLayer.backward()

In [23]:
! pytest --pyargs a4_course_cvdl_t1.tests.test_0relu -x

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: D:\edu\teach\course_cvdl\classes\c01
plugins: anyio-3.3.2
collected 4 items

. F

================================== FAILURES ===================================
_______________________ TestLayer.test_init[test_data0] _______________________

self = <a4_course_cvdl_t1.tests.base.TestLayer object at 0x000001EB467708B0>
test_data = {'meta': {'description': 'The only ReLU test.', 'layer_kwargs': {}}, 'values': {'input': [[[[0.3445909917354584, -0.44...36354065], [0.320019006729126, 0.7765920162200928, 0.8497490286827087, 0.796563982963562, 0.9298689961433411]]]], ...}}
test_layer_cls = <class 'a4_course_cvdl_t1.relu.ReluLayer'>

    def test_init(self, test_data, test_layer_cls):
        values = test_data['values']
        meta = test_data['meta']
        layer_kwargs = meta['layer_kwargs']
    
        # check layer creation
     

Исправляем в форке и проверяем баллы после теста

### Broadcasting

In [24]:
# Пусть у нас есть куча 2D точек
a = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
])

# И мы хотим их все "сдвинуть" на 2D вектор-константу
b = np.array([10, 20])

print(a.shape, b.shape)

(4, 2) (2,)


In [25]:
# Работает!
a + b

array([[10, 21],
       [12, 23],
       [14, 25],
       [16, 27]])

In [26]:
# Попробуем теперь каждой точке добавить разный сдвиг
b = np.array([10, 20, 30, 40])
print(a.shape, b.shape)
print("Broadcast error")
print("---------------")
a + b


(4, 2) (4,)
Broadcast error
---------------


ValueError: operands could not be broadcast together with shapes (4,2) (4,) 

In [27]:
# Приводим тензоы к явной форме для бродкаста
print(a.shape, b[:, None].shape)

a * b[:, None]

(4, 2) (4, 1)


array([[  0,  10],
       [ 40,  60],
       [120, 150],
       [240, 280]])

In [28]:
a = np.array([
    [0, 1],
    [1, 0],
    [1, 1]
])
b = np.array([
    [10, 20],
    [30, 40],
    [50, 60],
    [70, 80]
])
c = a[None] * b[:, None]
print("c[i, j, k] = a[1, i, k] * b[j, 1, k]")
print("------------------------------")
print(a.shape, b.shape, c.shape)

print(c)

c[i, j, k] = a[1, i, k] * b[j, 1, k]
------------------------------
(3, 2) (4, 2) (4, 3, 2)
[[[ 0 20]
  [10  0]
  [10 20]]

 [[ 0 40]
  [30  0]
  [30 40]]

 [[ 0 60]
  [50  0]
  [50 60]]

 [[ 0 80]
  [70  0]
  [70 80]]]


### Tensordot

In [29]:
help(np.tensordot)

Help on function tensordot in module numpy:

tensordot(a, b, axes=2)
    Compute tensor dot product along specified axes.
    
    Given two tensors, `a` and `b`, and an array_like object containing
    two array_like objects, ``(a_axes, b_axes)``, sum the products of
    `a`'s and `b`'s elements (components) over the axes specified by
    ``a_axes`` and ``b_axes``. The third argument can be a single non-negative
    integer_like scalar, ``N``; if it is such, then the last ``N`` dimensions
    of `a` and the first ``N`` dimensions of `b` are summed over.
    
    Parameters
    ----------
    a, b : array_like
        Tensors to "dot".
    
    axes : int or (2,) array_like
        * integer_like
          If an int N, sum over the last N axes of `a` and the first N axes
          of `b` in order. The sizes of the corresponding axes must match.
        * (2,) array_like
          Or, a list of axes to be summed over, first sequence applying to `a`,
          second to `b`. Both element

In [30]:
a = np.array([
    [0.5, -0.5],
    [-0.5, 0.5],
    [1, 0.]
])
b = np.array([10, 1])
print("c[i, k] = Sum(a[i, j] * b[j, k]")
print('-------------------------------')
print(a.shape, b.shape)
a @ b

c[i, k] = Sum(a[i, j] * b[j, k]
-------------------------------
(3, 2) (2,)


array([ 4.5, -4.5, 10. ])

In [31]:
np.tensordot(
    a, b,
    axes=[1, 0]
)

array([ 4.5, -4.5, 10. ])

Многомерный tensordot

In [32]:
a = np.arange(2* 3 *4).reshape((2, 3 ,4))
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [33]:
b = np.arange(4 * 2).reshape((4, 2)) % 2
b

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=int32)

In [34]:
np.tensordot(
    a, b,
    axes=[
        [-1, 0],
        [0, 1]
    ]
)

array([54, 70, 86])

### View vs Copy

In [35]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[1:, 1:, 1:]
b += 0.33
print(b)
print("---------")
print(a)

[[[17.33 18.33 19.33]
  [21.33 22.33 23.33]]]
---------
[[[ 0.    1.    2.    3.  ]
  [ 4.    5.    6.    7.  ]
  [ 8.    9.   10.   11.  ]]

 [[12.   13.   14.   15.  ]
  [16.   17.33 18.33 19.33]
  [20.   21.33 22.33 23.33]]]


In [36]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[1, 1, 1]
b += 0.33
print(b)
print("---------")
print(a)

17.33
---------
[[[ 0.  1.  2.  3.]
  [ 4.  5.  6.  7.]
  [ 8.  9. 10. 11.]]

 [[12. 13. 14. 15.]
  [16. 17. 18. 19.]
  [20. 21. 22. 23.]]]


In [37]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[None, 1:, 1:, 1:]
b += 0.33
print(b)
print("---------")
print(a)

[[[[17.33 18.33 19.33]
   [21.33 22.33 23.33]]]]
---------
[[[ 0.    1.    2.    3.  ]
  [ 4.    5.    6.    7.  ]
  [ 8.    9.   10.   11.  ]]

 [[12.   13.   14.   15.  ]
  [16.   17.33 18.33 19.33]
  [20.   21.33 22.33 23.33]]]


In [38]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
a[[0,0,0],[0, 1, 2], [-1, -1, -1]] += 0.33
print(a)

[[[ 0.    1.    2.    3.33]
  [ 4.    5.    6.    7.33]
  [ 8.    9.   10.   11.33]]

 [[12.   13.   14.   15.  ]
  [16.   17.   18.   19.  ]
  [20.   21.   22.   23.  ]]]


In [39]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[[0,0,0],[0, 1, 2], [-1, -1, -1]] 
b += 0.33
print(a)

[[[ 0.  1.  2.  3.]
  [ 4.  5.  6.  7.]
  [ 8.  9. 10. 11.]]

 [[12. 13. 14. 15.]
  [16. 17. 18. 19.]
  [20. 21. 22. 23.]]]
